### NLM CUDA

In [5]:
import skimage.io
import skimage.color
import cupy as cp
import time

# Função para replicar bordas (mirror padding)
def mirror(A, f):
    n, m = A.shape
    nlin = n + 2*f
    ncol = m + 2*f
    B = cp.zeros((nlin, ncol), dtype=A.dtype)
    B[f:nlin-f, f:ncol-f] = A
    B[0:f, 0:f] = cp.flip(A[0:f, 0:f])
    B[0:f, ncol-f:ncol] = cp.flip(A[0:f, m-f:m])
    B[nlin-f:nlin, 0:f] = cp.flip(A[n-f:n, 0:f])
    B[nlin-f:nlin, ncol-f:ncol] = cp.flip(A[n-f:n, m-f:m])
    B[0:f, f:ncol-f] = cp.flipud(A[0:f, :])
    B[nlin-f:nlin, f:ncol-f] = cp.flipud(A[n-f:n, :])
    B[f:nlin-f, 0:f] = cp.fliplr(A[:, 0:f])
    B[f:nlin-f, ncol-f:ncol] = cp.fliplr(A[:, m-f:m])
    return B

nlm_kernel_shared_code = r'''
extern "C" __global__
void nlm_kernel_shared(
    const float* img_n, float* output,
    int m, int n, int f, int t, float h, int padded_width
) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    int Bx = blockDim.x;
    int By = blockDim.y;

    int pad = f + t;

    int sh_width = Bx + 2 * pad;
    int sh_height = By + 2 * pad;

    extern __shared__ float sh_img[];

    int base_i = blockIdx.y * blockDim.y + f - pad;
    int base_j = blockIdx.x * blockDim.x + f - pad;

    // Carrega patch expandido na shared memory
    for (int y = threadIdx.y; y < sh_height; y += By) {
        for (int x = threadIdx.x; x < sh_width; x += Bx) {
            int img_i = base_i + y;
            int img_j = base_j + x;

            // Replicação de borda
            int ii = img_i < 0 ? 0 : (img_i >= m + 2*f ? m + 2*f - 1 : img_i);
            int jj = img_j < 0 ? 0 : (img_j >= n + 2*f ? n + 2*f - 1 : img_j);

            sh_img[y * sh_width + x] = img_n[ii * padded_width + jj];
        }
    }
    __syncthreads();

    if (i >= m || j >= n) return;

    int local_i = threadIdx.y + pad;
    int local_j = threadIdx.x + pad;

    float NL = 0.0f;
    float Z = 0.0f;

    int rmin = max(local_i - t, pad);
    int rmax = min(local_i + t, By + pad - 1);
    int smin = max(local_j - t, pad);
    int smax = min(local_j + t, Bx + pad - 1);

    for (int r = rmin; r <= rmax; ++r) {
        for (int s = smin; s <= smax; ++s) {
            float d2 = 0.0f;
            for (int u = -f; u <= f; ++u) {
                for (int v = -f; v <= f; ++v) {
                    float diff = sh_img[(local_i + u) * sh_width + (local_j + v)] -
                                 sh_img[(r + u) * sh_width + (s + v)];
                    d2 += diff * diff;
                }
            }
            float sij = __expf(-d2 / (h * h));
            Z += sij;
            NL += sij * sh_img[r * sh_width + s];
        }
    }
    output[i * n + j] = NL / Z;
}
'''

def NLM_fast_cuda_shared(img, h, f, t):
    img = img.astype(cp.float32)
    m, n = img.shape
    padded = mirror(img, f)

    kernel_code = nlm_kernel_shared_code.encode('ascii', 'ignore').decode('ascii')
    module = cp.RawModule(code=kernel_code, options=('-std=c++11',))
    #module = cp.RawModule(code=nlm_kernel_shared_code, options=('-std=c++11',))
    kernel = module.get_function("nlm_kernel_shared")

    output = cp.zeros((m, n), dtype=cp.float32)

    threads_per_block = (16, 16)
    block_x = (n + threads_per_block[0] - 1) // threads_per_block[0]
    block_y = (m + threads_per_block[1] - 1) // threads_per_block[1]
    grid = (block_x, block_y)

    sh_width = threads_per_block[0] + 2 * (f + t)
    sh_height = threads_per_block[1] + 2 * (f + t)
    shared_mem_size = sh_width * sh_height * 4  # float32 = 4 bytes

    kernel(
        grid, threads_per_block,
        (
            padded.ravel(), output.ravel(),
            cp.int32(m), cp.int32(n), cp.int32(f), cp.int32(t),
            cp.float32(h), cp.int32(padded.shape[1])
        ),
        shared_mem=shared_mem_size
    )

    return output

### GEO NLM

In [4]:
import sys
import warnings
import time
import skimage
import statistics
import networkx as nx
import matplotlib.pyplot as plt
import skimage.io
import skimage.measure
import numpy as np
import sklearn.neighbors as sknn
from scipy.sparse.csgraph import dijkstra
from numpy.matlib import repmat
from scipy.linalg import eigh
from numpy.linalg import inv
from numpy.linalg import cond
from numpy import eye
from sklearn.decomposition import PCA
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity
from skimage.transform import rescale, resize, downscale_local_mean
from numba import njit   # just in time compiler (acelera loops)
from joblib import Parallel, delayed
from bm3d import bm3d, BM3DProfile

# Para evitar warning de divisão por zero
warnings.simplefilter(action='ignore')


'''
Non-Local Means geodésico (versão básica, sem paralelismo e mais lenta)

Parâmetros:

    img: imagem ruidosa de entrada
    h: parâmetro que controla o grau de suavização (quanto maior, mais suaviza)
    f: tamanho do patch (2f + 1 x 2f + 1) -> se f = 3, então patch é 7 x 7
    t: tamanho da janela de busca (2t + 1 x 2t + 1) -> se t = 10, então janela de busca é 21 x 21
    nn: número de vizinhos no grafo KNN

''' 
def GeoNLM(img_noise, h, f, t, nn=10):
    # Dimenssões espaciais da imagem
    m, n = img_noise.shape
    # Cria imagem de saída
    filtrada = np.zeros((m, n))
    # Problema de valor de contorno: replicar bordas
    img_n = np.pad(img_noise, ((f, f), (f, f)), 'symmetric')
    # Loop principal do NLM geodésico
    for i in range(m):
        if i % 10 == 0:
            print(i, end=' ')
            sys.stdout.flush()
        for j in range(n):
            im = i + f;   # compensar a borda adicionada artificialmente
            jn = j + f;   # compensar a borda adicionada artificialmente
            # Obtém o patch ao redor do pixel corrente
            patch_central = img_n[im-f:(im+f)+1, jn-f:(jn+f)+1]
            central = np.reshape(patch_central, [1, patch_central.shape[0]*patch_central.shape[1]])[-1]
            # Calcula as bordas da janela de busca para o pixel corrente
            rmin = max(im-t, f);  # linha inicial
            rmax = min(im+t, m+f);  # linha final
            smin = max(jn-t, f);  # coluna inicial
            smax = min(jn+t, n+f);  # coluna final
            # Calcula média ponderada
            NL = 0      # valor do pixel corrente filtrado
            Z = 0       # constante normalizadora
            # Cria dataset com patches da janela de busca como vetores
            num_elem = (rmax - rmin)*(smax - smin)
            tamanho_patch = (2*f + 1)*(2*f + 1)
            dataset = np.zeros((num_elem, tamanho_patch))
            k = 0
            pixels_busca = []
            # Loop para montar o dataset com todos os patches da janela
            for r in range(rmin, rmax):
                for s in range(smin, smax):
                    # Obtém o patch ao redor do pixel a ser comparado
                    W = img_n[r-f:(r+f)+1, s-f:(s+f)+1] 
                    neighbor = np.reshape(W, [1, W.shape[0]*W.shape[1]])[-1]                    
                    dataset[k, :] = neighbor.copy()
                    if central[0] == neighbor[0]:
                        if (central == neighbor).all():
                            source = k
                    pixels_busca.append(img_n[r, s])
                    k = k + 1
            # Cria grafo knn com patches da janela de busca
            knnGraph = sknn.kneighbors_graph(dataset, n_neighbors=nn, mode='distance')
            A = knnGraph.toarray()
            # Converte matriz de adjacências para grafo
            G = nx.from_numpy_array(A)                      
            # Aplica algoritmo de Dijkstra
            length, path = nx.single_source_dijkstra(G, source)
            points = np.array(list(length.keys()))
            distancias = np.array(list(length.values()))
            # Calcula similaridades
            similaridades = np.exp(-distancias**2/(h**2))
            pixels = np.zeros(len(points))
            pixels_busca = np.array(pixels_busca)
            pixels = pixels_busca[points]
            # Normalização do pixel filtrado
            NL = sum(similaridades*pixels)
            Z = sum(similaridades)
            filtrada[i, j] = NL/Z
    return filtrada


######################################################
# Função auxiliar para paralelizar o GEONLM
######################################################
def process_pixel(i, j, img_n, f, t, h, nn):
    im = i + f
    jn = j + f
    patch_central = img_n[im-f:(im+f)+1, jn-f:(jn+f)+1]
    central = np.reshape(patch_central, [1, patch_central.shape[0]*patch_central.shape[1]])[-1]
    rmin = max(im-t, f)
    rmax = min(im+t, m+f)
    smin = max(jn-t, f)
    smax = min(jn+t, n+f)
    NL, Z = 0, 0
    dataset = np.zeros(((rmax - rmin)*(smax - smin), (2*f + 1)*(2*f + 1)))
    k = 0
    pixels_busca = []
    for r in range(rmin, rmax):
        for s in range(smin, smax):
            W = img_n[r-f:(r+f)+1, s-f:(s+f)+1]
            neighbor = np.reshape(W, [1, W.shape[0]*W.shape[1]])[-1]
            dataset[k, :] = neighbor.copy()
            if central[0] == neighbor[0] and (central == neighbor).all():
                source = k
            pixels_busca.append(img_n[r, s])
            k += 1
    knnGraph = sknn.kneighbors_graph(dataset, n_neighbors=nn, mode='distance')
    A = knnGraph.toarray()
    G = nx.from_numpy_array(A)
    length, path = nx.single_source_dijkstra(G, source)
    points = np.array(list(length.keys()))
    distancias = np.array(list(length.values()))
    similaridades = np.exp(-distancias**2 / (h**2))
    pixels_busca = np.array(pixels_busca)
    pixels = pixels_busca[points]
    NL = sum(similaridades * pixels)
    Z = sum(similaridades)
    return NL / Z

##################################################
# GEONLM paralelo 
##################################################
def Parallel_GEONLM(img_n, f, t, h, nn):
    # Parallelize the loop
    print(f'img_n.shape: {img_n.shape}')
    m = img_n.shape[0] - 2*f
    print(f'M: {m}')
    n = img_n.shape[1] - 2*f
    print(f'N: {n}')
    filtrada = Parallel(n_jobs=-1)(delayed(process_pixel)(i, j, img_n, f, t, h, nn) for i in range(m) for j in range(n))
    #print(f"filtrada Shape: {filtrada.shape}")
    
    filtrada_geo = np.array(filtrada).reshape((m, n))
    #print(f'filtrada_geo.shape: {filtrada_geo.shape}')
    return filtrada_geo

####################################################################
'''
Função que extrai os patches de cada janela de busca no GeoNLM
Retorna uma matriz 4D (m, n, 2t+1 x 2t+1, 2f+1 x 2f+1)

Usa o JIT (just in time) compiler para acelerar loops

'''
####################################################################
@njit
def Extract_patches(img, f, t):
    # Dimenssões espaciais da imagem
    m, n = img.shape
    # Tamanhos do patch e da janela de busca
    tamanho_patch = (2*f + 1)*(2*f + 1)    
    # Patches para cada janela de busca
    patches = []
    centros = []    
    # Problema de valor de contorno: replicar bordas
    img_n = mirror(img, f)
    # Loop principal do NLM geodésico
    for i in range(m):        
        for j in range(n):
            im = i + f;   # compensar a borda adicionada artificialmente
            jn = j + f;   # compensar a borda adicionada artificialmente
            # Obtém o patch ao redor do pixel corrente
            patch_central = img_n[im-f:(im+f)+1, jn-f:(jn+f)+1].copy()
            central = patch_central.reshape((1, patch_central.shape[0]*patch_central.shape[1]))[-1]
            # Calcula as bordas da janela de busca para o pixel corrente
            rmin = max(im-t, f);  # linha inicial
            rmax = min(im+t, m+f);  # linha final
            smin = max(jn-t, f);  # coluna inicial
            smax = min(jn+t, n+f);  # coluna final
            num_elem = (rmax - rmin)*(smax - smin)
            # Cria dataset
            dataset = np.zeros((num_elem, tamanho_patch))
            # Loop para montar o dataset com todos os patches da janela
            k = 0
            for r in range(rmin, rmax):
                for s in range(smin, smax):
                    # Obtém o patch ao redor do pixel a ser comparado
                    W = img_n[r-f:(r+f)+1, s-f:(s+f)+1].copy() 
                    neighbor = W.reshape((1, W.shape[0]*W.shape[1]))[-1]
                    dataset[k, :] = neighbor.copy()
                    if (central == neighbor).all():
                        source = k
                    k = k + 1
            patches.append(dataset)
            centros.append(source)
    return patches, centros

###################################################################
'''
Função que extrai os patches de cada janela de busca no GeoNLM
Retorna uma lista 
'''
###################################################################
def Geodesic_distances(patches, centros, nn=10):
    distancias_geodesicas = []
    pontos = []
    # Percorre a lista de patches
    for i in range(len(patches)):
    # Cria grafo knn com patches da janela de busca
        knnGraph = sknn.kneighbors_graph(patches[i], n_neighbors=nn, mode='distance')
        A = knnGraph.toarray()        
        G = nx.from_numpy_array(A)      # Converte matriz de adjacências para grafo
        # Aplica algoritmo de Dijkstra
        length, path = nx.single_source_dijkstra(G, centros[i])
        points = np.array(list(length.keys()))
        pontos.append(points)
        geodists = np.array(list(length.values()))        
        distancias_geodesicas.append(geodists)
    return distancias_geodesicas, pontos

##################################################################################################
'''
Non-Local Means geodésico (versão com compilador JIT para acelerar loops)

Parâmetros:

    img: imagem ruidosa de entrada
    h: parâmetro que controla o grau de suavização (quanto maior, mais suaviza)
    f: tamanho do patch (2f + 1 x 2f + 1) -> se f = 3, então patch é 7 x 7
    t: tamanho da janela de busca (2t + 1 x 2t + 1) -> se t = 10, então janela de busca é 21 x 21
    nn: número de vizinhos no grafo KNN

''' 
###################################################################################################
@njit
def GeoNLM_fast(img, h, f, t, distancias_geodesicas, pontos):
    # Dimenssões espaciais da imagem
    m, n = img.shape
    # Cria imagem de saída
    filtrada = np.zeros((m, n))
    # Problema de valor de contorno: replicar bordas
    #img_n = np.pad(ruidosa, ((f, f), (f, f)), 'symmetric')
    img_n = mirror(img, f)
    # Loop principal do NLM geodésico
    k = 0
    for i in range(m):
        for j in range(n):
            im = i + f;   # compensar a borda adicionada artificialmente
            jn = j + f;   # compensar a borda adicionada artificialmente    
            # Calcula as bordas da janela de busca para o pixel corrente
            rmin = max(im-t, f);  # linha inicial
            rmax = min(im+t, m+f);  # linha final
            smin = max(jn-t, f);  # coluna inicial
            smax = min(jn+t, n+f);  # coluna final
            # Calcula média ponderada
            NL = 0      # valor do pixel corrente filtrado
            Z = 0       # constante normalizadora            
            pixels_busca = []
            # Loop para montar o dataset com todos os patches da janela
            for r in range(rmin, rmax):
                for s in range(smin, smax):
                    # Obtém o patch ao redor do pixel a ser comparado
                    pixels_busca.append(img_n[r, s])
            # Calcula similaridades
            similaridades = np.exp(-distancias_geodesicas[k]**2/(h**2))
            pixels = np.zeros(len(pontos[k]))
            pixels_busca = np.array(pixels_busca)
            pixels = pixels_busca[pontos[k]]
            # Normalização do pixel filtrado
            NL = sum(similaridades*pixels)
            Z = sum(similaridades)
            filtrada[i, j] = NL/Z
            k = k + 1
    return filtrada

#########################################################################
'''
Realiza a filtragem da imagem com o filtro NLM geodésico 

Usa o compilador JIT para acelerar loops
'''
#########################################################################
def GeoNLM_filter(img_noise, h, f, t, nn=10):
    # Fase 1
    print('Início da extração dos patches')
    inicio = time.time()
    patches, centros = Extract_patches(img_noise, f, t)
    fim = time.time()
    print('Elapsed time: %f ' %(fim - inicio))
    print()
    # Fase 2
    print('Início do cálculo das distâncias')
    inicio = time.time()
    distancias_geodesicas, pontos = Geodesic_distances(patches, centros, nn)
    fim = time.time()
    print('Elapsed time: %f ' %(fim - inicio))
    print()
    # Fase 3
    print('Início da filtragem')
    inicio = time.time()
    filtrada = GeoNLM_fast(img_noise, h, f, t, distancias_geodesicas, pontos)
    fim = time.time()
    print('Elapsed time: %f ' %(fim - inicio))
    return filtrada


### Funções

In [19]:
import os
import math
import numpy as np
import cupy as cp

def read_directories(directory, img=None, exclude_json=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)
        elif exclude_json is not None:
            filenames.append(filename.replace('.json',''))     
        else:
            filenames.append(filename)    
    return filenames


def add_poisson_noise(img):
    """
    Aplica ruído de Poisson corretamente sem overflow, utilizando CuPy (GPU).

    Parâmetros:
        img (cp.ndarray): Imagem com valores em [0,255] ou [0,1].

    Retorna:
        cp.ndarray: imagem ruidosa, clipada para [0, 255], dtype uint8.
    """
    # Se estiver em [0, 1], escala para 0-255
    if cp.max(img) <= 1.0:
        img = (img * 255).astype(cp.float32)
    else:
        img = img.astype(cp.float32)

    # Garante que os valores Poisson não causem overflow
    poisson_img = cp.random.poisson(img).astype(cp.float32)
    poisson_img = cp.clip(poisson_img, 0, 255)

    return poisson_img.astype(cp.uint8)


def anscombe_transform(img):
    return 2.0 * cp.sqrt(img.astype(cp.float32) + 3.0 / 8.0)

def inverse_anscombe(transf_img):
    return cp.clip((transf_img / 2.0) ** 2 - 3.0 / 8.0, 0, 255)

def compute_adaptive_q(sigma_est):
    q_nlm = 0.8 + 0.5 * cp.tanh(0.3 * (sigma_est - 1))
    q_geo = 1.0 + 0.7 * cp.tanh(0.25 * (sigma_est - 1.5))

    q_nlm = cp.clip(q_nlm, 0.7, 2.2) * 10
    q_geo = cp.clip(q_geo, 0.9, 2.7) * 10

    return q_nlm, q_geo

import cupy as cp

def add_poisson_gaussian_noise(image, gaussian_sigma=25):
    """
    Adds Poisson noise followed by Gaussian noise to an image.

    Parameters:
        image (cp.ndarray): Input image (grayscale or RGB), values in [0, 1] or [0, 255].
        gaussian_sigma (float): Standard deviation of the Gaussian noise (in intensity scale 0–255).

    Returns:
        cp.ndarray: Noisy image (clipped to [0, 255] and converted to uint8).
    """
    # Convert to float32 and normalize to [0, 255] if necessary
    if image.dtype != cp.float32:
        image = image.astype(cp.float32)

    if image.max() <= 1.0:
        image *= 255.0

    # Apply Poisson noise
    poisson_image = cp.random.poisson(image).astype(cp.float32)

    # Apply Gaussian noise
    gaussian_noise = cp.random.normal(loc=0.0, scale=gaussian_sigma, size=image.shape)
    noisy_image = poisson_image + gaussian_noise

    # Clip values and convert to uint8
    noisy_image = cp.clip(noisy_image, 0, 255).astype(cp.uint8)

    return noisy_image


In [26]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.restoration import estimate_sigma
import cupy as cp


def select_best_h_using_adaptive_q(img_original, img_ruidosa, f, t, alpha=0.5):
    """
    Seleciona o melhor h para NLM com base em pequenas variações de q_nlm,
    utilizando a função compute_adaptive_q, com critério combinado PSNR e SSIM.

    Parâmetros:
    - img_original: imagem original sem ruído (uint8)
    - img_ruidosa: imagem ruidosa (uint8)
    - f, t: parâmetros do filtro NLM
    - alpha: peso da PSNR (0 a 1). O restante (1-alpha) será o peso da SSIM

    Retorna:
    - h_nlm_final: melhor h para NLM
    - h_geo_final: valor adaptado para GEONLM
    """

    img_ruidosa_asc = anscombe_transform(img_ruidosa)
    sigma_est = estimate_sigma(img_ruidosa_asc)
    q_nlm_base, q_geo_base = compute_adaptive_q(sigma_est)

    # Gera variações do q_nlm base com delta
    q_nlm_candidates = cp.array([q_nlm_base + delta for delta in range(-20, 25, 5)])

    melhor_score = -cp.inf
    melhor_q_nlm = None

    for q in q_nlm_candidates:
        h_nlm = q * sigma_est
        img_filt_asc = NLM_fast_cuda_shared(img_ruidosa_asc, h_nlm, f, t)
        img_filt = inverse_anscombe(img_filt_asc).astype(cp.uint8)

        # Calculando PSNR e SSIM
        psnr = peak_signal_noise_ratio(cp.asnumpy(img_original), cp.asnumpy(img_filt))
        ssim = structural_similarity(cp.asnumpy(img_original), cp.asnumpy(img_filt))

        # Calculando o score combinado
        score = alpha * psnr + (1 - alpha) * (ssim * 100)  # normaliza SSIM para escala próxima ao PSNR

        print(f"q = {q:.2f} | h = {h_nlm:.2f} | PSNR = {psnr:.2f} | SSIM = {ssim:.4f} | Score = {score:.2f}")

        if score > melhor_score:
            melhor_score = score
            melhor_q_nlm = q

    h_nlm_final = melhor_q_nlm * sigma_est
    h_geo_final = (melhor_q_nlm + 0.3) * sigma_est  # leve ajuste para mais suavização no GEONLM

    print(f"\n[SELECIONADO] q_nlm = {melhor_q_nlm:.2f} | h_nlm = {h_nlm_final:.2f} | h_geo = {h_geo_final:.2f}")

    return h_nlm_final, h_geo_final, q_nlm_candidates



In [27]:
import cupy as cp
import skimage.io
import skimage.color
from skimage.restoration import estimate_sigma
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from pathlib import Path

# Função para downscale com CuPy
def downscale_with_cp(image, scale_factor):
    """
    Aplica downscale (média local) usando CuPy, redimensionando a imagem.
    O downscale é feito aplicando a média sobre blocos.

    Parâmetros:
        image (cp.ndarray): imagem a ser reduzida (CuPy array).
        scale_factor (float): fator de escala para downscale (deve ser um número positivo).

    Retorna:
        cp.ndarray: imagem com downscale (CuPy array).
    """
    # Verifica se a imagem é 2D ou 3D
    if len(image.shape) == 2:
        m, n = image.shape
    elif len(image.shape) == 3:
        m, n, c = image.shape
    else:
        raise ValueError("A imagem deve ter 2 ou 3 dimensões (escala de cinza ou RGB).")

    block_size = int(scale_factor)

    # Verificar se o tamanho é divisível pelo block_size
    if m % block_size != 0 or n % block_size != 0:
        raise ValueError("O tamanho da imagem não é divisível pelo block_size.")

    # Divisão da imagem em blocos e calculando a média
    image_reshaped = image.reshape(m // block_size, block_size, n // block_size, block_size)
    downscaled = image_reshaped.mean(axis=(1, 3))  # Calculando a média de cada bloco

    return downscaled

# Carregar a imagem
img = skimage.io.imread('../images/0.gif')

# Se for uma imagem com múltiplos quadros (por exemplo, GIF animado), pegamos o primeiro
if img.ndim > 2 and img.shape[0] > 1:
    img = img[0, :, :]

# Se a imagem for colorida (RGB), converta para monocromática (escala de cinza)
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)  # valores convertidos ficam entre 0 e 1
    img = (255 * img).astype(cp.float32)  # Converte para [0,255]

# Redimensionar a imagem (downscale com CuPy)
img = cp.array(img).astype(cp.float32)
img_downscaled = downscale_with_cp(img, 2)  # Aplicar downscale com fator de escala 2

m, n = img_downscaled.shape

# Adicionar ruído de Poisson
noised_poisson = add_poisson_noise(img_downscaled)

# Adicionar ruído Poisson + Gaussiano
noised_poissongaussian = add_poisson_gaussian_noise(img_downscaled)

# Garantir que as imagens de ruído fiquem entre 0 e 255
noised_poisson[cp.where(noised_poisson > 255)] = 255
noised_poisson[cp.where(noised_poisson < 0)] = 0

noised_poissongaussian[cp.where(noised_poissongaussian > 255)] = 255
noised_poissongaussian[cp.where(noised_poissongaussian < 0)] = 0

# Selecionar o melhor h para NLM e GEONLM
h_nlm, h_geo, q_nlm_candidates = select_best_h_using_adaptive_q(img_downscaled, noised_poisson, f=4, t=7, alpha=0.5)

print(f'h_nlm: {h_nlm}, h_geo: {h_geo}, q_nlm_candidates: {q_nlm_candidates}')



# f = 4
# t = 7
# # Cria imagem de saída
# filtered = np.zeros((m, n))

# # Problema de valor de contorno: replicar bordas
# img_noised_poisson = np.pad(noised_poisson, ((f, f), (f, f)), 'symmetric')
# img_noised_poisson_gaussian = np.pad(noised_poissongaussian, ((f, f), (f, f)), 'symmetric')

# noised_anscombe_poisson = anscombe_transform(img_noised_poisson)
# noised_anscombe_poisson_gaussian = anscombe_transform(img_noised_poisson_gaussian)


# sigma_est_poisson = estimate_sigma(noised_anscombe_poisson)
# sigma_est_poisson_gaussian = estimate_sigma(noised_anscombe_poisson_gaussian)

# q_nlm, q_geo = compute_adaptive_q(sigma_est)
# h_nlm = q_nlm * sigma_est #* 10
# h_geo = q_geo * sigma_est #* 10


# start = time.time()
# result_img_noised_poisson = NLM_fast_cuda_shared(img_noised_poisson, h, f, t)
# result_img_noised_poisson_gaussian = NLM_fast_cuda_shared(img_noised_poisson_gaussian, h, f, t)

cp.cuda.Stream.null.synchronize()  # espera terminar GPU
#print("Tempo GPU:", time.time()-start)


ValueError: O tamanho da imagem não é divisível pelo block_size.